# Advent of Code Day 23
#### Problem in full can be found here: https://adventofcode.com/2023/day/23

In [24]:
# Part one was using backtracking to explore all options but since the input was too big,
# it prints the max as it calculates and after a while, the answer is printed
# Part 1: has slopes '>' '<' 'v' '^' where it forces one direction
# Part 2: remove the slopes

# Get the input in a map
file = open("inputday23.txt")
map = []
for line in file:
    line = line.strip()
    map.append(list(line))
file.close()

# Start and end coordinates
start = (0, 1)
end = (len(map[0])-1, len(map)-2)

print(findHikes(set(), start, end, 0, False))

5127
5475
5487
5543
5591
5787
5835
6023
6035
6207


KeyboardInterrupt: 

In [21]:
# A function that finds a path and returns the max length of steps for the path
def findHikes(hike, current, end, currentMax, slopes=True):  
    hike.add(current)
    
    if current == end:
        if len(hike) > currentMax:
            currentMax = len(hike)
            print(currentMax)
        return currentMax

    possiblePaths = findPaths(current, hike, slopes)
    if len(possiblePaths) == 0:
        return currentMax
    else:
        for path in possiblePaths:
            currentMax = findHikes(hike.copy(), path, end, currentMax, slopes)

    return currentMax

In [19]:
# A function that given the current point, finds all possible paths it can go from the current point
def findPaths(current, hike, slopes):
    possiblePaths = []

    rows = len(map)
    cols = len(map[0])

    # Cases where there is a symbol that forces a specific path
    if slopes:
        symbol = map[current[0]][current[1]]
        if symbol == '^':
            return [(current[0]-1, current[1])] if current[0] > 0 else []
        elif symbol == 'v':
            return [(current[0]+1, current[1])] if current[0] < rows - 1 else []
        elif symbol == '>':
            return [(current[0], current[1]+1)] if current[1] < cols - 1 else []
        elif symbol == '<':
            return [(current[0], current[1]-1)] if current[1] > 0 else []

    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    for dir in directions:
        newX, newY = current[0] + dir[0], current[1] + dir[1]
        if 0 <= newX < rows and 0 <= newY < cols and map[newX][newY] != '#' and (newX, newY) not in hike:
            possiblePaths.append((newX, newY))

    return possiblePaths

In [23]:
import sys
sys.setrecursionlimit(100000)

In [36]:
import sys
from collections import defaultdict, deque

def neighbors(grid, r, c, ignore_slopes):
	cell = grid[r][c]

	if ignore_slopes or cell == '.':
		for r, c in ((r + 1, c), (r - 1, c), (r, c + 1), (r, c - 1)):
			if grid[r][c] != '#':
				yield r, c
	elif cell == 'v': yield (r + 1, c)
	elif cell == '^': yield (r - 1, c)
	elif cell == '>': yield (r, c + 1)
	elif cell == '<': yield (r, c - 1)

def num_neighbors(grid, r, c, ignore_slopes):
	if ignore_slopes or grid[r][c] == '.':
		return sum(grid[r][c] != '#' for r, c in ((r + 1, c), (r - 1, c), (r, c + 1), (r, c - 1)))
	return 1

def is_node(grid, rc, src, dst, ignore_slopes):
	return rc == src or rc == dst or num_neighbors(grid, *rc, ignore_slopes) > 2

def adjacent_nodes(grid, rc, src, dst, ignore_slopes):
	q = deque([(rc, 0)])
	seen = set()

	while q:
		rc, dist = q.popleft()
		seen.add(rc)

		for n in neighbors(grid, *rc, ignore_slopes):
			if n in seen:
				continue

			if is_node(grid, n, src, dst, ignore_slopes):
				yield (n, dist + 1)
				continue

			q.append((n, dist + 1))

def graph_from_grid(grid, src, dst, ignore_slopes=False):
	g = defaultdict(list)
	q = deque([src])
	seen = set()

	while q:
		rc = q.popleft()
		if rc in seen:
			continue

		seen.add(rc)

		for n, weight in adjacent_nodes(grid, rc, src, dst, ignore_slopes):
			g[rc].append((n, weight))
			q.append(n)

	return g

def longest_path(g, cur, dst, distance=0, seen=set()):
	if cur == dst:
		return distance

	best = 0
	seen.add(cur)

	for neighbor, weight in g[cur]:
		if neighbor in seen:
			continue

		best = max(best, longest_path(g, neighbor, dst, distance + weight))

	seen.remove(cur)
	return best


# Open the first argument as input or use stdin if no arguments were given
fin = open("inputday23.txt")

grid = []
for line in fin:
    line = line.strip()
    grid.append(list(line))
height, width = len(grid), len(grid[0])
fin.close()

grid[0][1] = '#'
grid[height - 1][width - 2] = '#'

src = (1, 1)
dst = (height - 2, width - 2)

g = graph_from_grid(grid, src, dst)
pathlen = longest_path(g, src, dst) + 2
print('Part 1:', pathlen)

g = graph_from_grid(grid, src, dst, ignore_slopes=True)
pathlen = longest_path(g, src, dst) + 2
print('Part 2:', pathlen)

Part 1: 2358
Part 2: 6586
